In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import csv

In [14]:
# Liste pour stocker les DataFrames
dataframes = []

# Parcourir tous les fichiers dans le dossier
files = [f for f in os.listdir() if f.endswith('.csv')]  # Liste des fichiers CSV

# Parcourir chaque fichier CSV
for fichier in files:
    chemin_fichier = os.path.join(os.getcwd(), fichier)
    rows = []  # Liste pour stocker les données du fichier courant
    
    # Tentative d'ouverture avec un encodage ISO-8859-1 (latin1)
    with open(chemin_fichier, mode="r", newline="", encoding="ISO-8859-1") as file:
        reader = csv.DictReader(file, delimiter=";")  # Adapter le délimiteur si nécessaire
        for row in reader:
            # Extraire les informations nécessaires
            level = row.get("level", "unknown") 
            score = int(row.get("score", 0))
            mode = row.get("mode", "unknown")
            cleared = row.get("cleared", "unknown")
            player_name = os.path.splitext(fichier)[0]
            
            # Ajouter une ligne au fichier courant
            rows.append({
                "player_name": player_name,
                "level": level,
                "score": score,
                "mode": mode,
                "cleared": cleared,  # Inclure la colonne "clear"
            })

    # Convertir les données en DataFrame
    df = pd.DataFrame(rows)
    dataframes.append(df)  # Ajouter ce DataFrame à la liste

# Vérifier si des DataFrames ont été ajoutés
for i, df in enumerate(dataframes):
    print(f"DataFrame {i + 1} :")
    print(df)
    print("\n" + "="*50 + "\n")



ValueError: invalid literal for int() with base 10: '214.0'

In [8]:
print(df)

   player_name                           level  score    mode cleared
0          vic         Intermediate - Warriors      5  Normal      No
1          vic         Intermediate - Warriors      3   Rythm      No
2          vic         Intermediate - Warriors      2  Normal      No
3          vic         Intermediate - Warriors      0  Normal      No
4          vic         Intermediate - Warriors      0  Normal      No
5          vic                   GOD - Camelia      0   Rythm      No
6          vic                   GOD - Camelia      0  Normal      No
7          vic                   GOD - Camelia      0  Normal      No
8          vic                   GOD - Camelia      1   Rythm      No
9          vic                   GOD - Camelia      0   Rythm      No
10         vic                   GOD - Camelia      0   Rythm      No
11         vic                   GOD - Camelia      0   Rythm      No
12         vic                   GOD - Camelia      0   Rythm      No
13         vic      

In [43]:
df1 = pd.read_csv('lelian.csv', delimiter=";")
print(df1)

                      level              date  score    mode cleared
0        Easy - Next To You  15/11/2024 15:19     21  Normal      No
1   Intermediate - Warriors  15/11/2024 15:19      1  Normal      No
2   Intermediate - Warriors  15/11/2024 15:19      1  Normal      No
3   Intermediate - Warriors  15/11/2024 15:19      1  Normal      No
4   Intermediate - Warriors  15/11/2024 15:20      0  Normal      No
5   Intermediate - Warriors  15/11/2024 15:20     18   Rythm      No
6             GOD - Camelia  15/11/2024 15:21      9  Normal      No
7             GOD - Camelia  15/11/2024 15:21      0   Rythm      No
8             GOD - Camelia  15/11/2024 15:21      0   Rythm      No
9             GOD - Camelia  15/11/2024 15:22      1  Normal      No
10            GOD - Camelia  15/11/2024 15:22      0  Normal      No
11            GOD - Camelia  15/11/2024 15:22      1  Normal      No
12            GOD - Camelia  15/11/2024 15:22      2  Normal      No
13        Hard - Beat Saber  15/11

In [9]:
import os
import pandas as pd

# Liste pour stocker les DataFrames
dataframes = []

# Dossier contenant les fichiers CSV
dossier = os.getcwd()  # Modifier si nécessaire

# Parcourir tous les fichiers CSV dans le dossier
files = [f for f in os.listdir(dossier) if f.endswith('.csv')]

# Lire chaque fichier CSV
for fichier in files:
    chemin_fichier = os.path.join(dossier, fichier)
    
    # Charger le fichier CSV
    df = pd.read_csv(chemin_fichier, delimiter=";", encoding="ISO-8859-1")
    
    # Vérifier et renommer la colonne de date pour un accès facile
    if "date" in df.columns:
        df.rename(columns={"date": "formatted_date"}, inplace=True)
        # Convertir la colonne 'date' en format datetime
        df['formatted_date'] = pd.to_datetime(df['formatted_date'], format="%d/%m/%Y %H:%M", errors="coerce")
    
    # Ajouter une colonne pour identifier la source (nom du fichier sans extension)
    df['source_file'] = os.path.splitext(fichier)[0]
    
    # Ajouter le DataFrame à la liste
    dataframes.append(df)

# Fusionner tous les DataFrames en un seul
resultat = pd.concat(dataframes, ignore_index=True)

# Afficher le DataFrame final
print(resultat)

# Optionnel : sauvegarder dans un fichier CSV unique
resultat.to_csv("resultat_combine.csv", index=False, sep=";")


                              level  \
0                Easy - Next To You   
1                Easy - Next To You   
2                 Hard - Beat Saber   
3                 Hard - Beat Saber   
4                 Hard - Beat Saber   
..                              ...   
117  Ultra Hardcore - Hardstyle LFN   
118  Ultra Hardcore - Hardstyle LFN   
119  Ultra Hardcore - Hardstyle LFN   
120                   GOD - Camelia   
121                   GOD - Camelia   

    date start (année mois jour heure minute seconde)  score    mode cleared  \
0                                 2024-11-15 16:16:13  214.0  Normal     Yes   
1                                 2024-11-15 16:16:22    0.0  Normal      No   
2                                 2024-11-15 16:16:42    3.0   Rythm      No   
3                                 2024-11-15 16:16:57    5.0  Normal      No   
4                                 2024-11-15 16:17:09    3.0  Normal      No   
..                                                .